In [1]:
using Revise
using PastaQ
using ITensors
using Random
using OptimKit
using Zygote
using Zygote: ChainRulesCore
using BenchmarkTools
using JLD
using MethodAnalysis
using Plots

In [2]:
@time begin
import mVQE
using mVQE.ITensorsExtension: projective_measurement
using mVQE.StateFactory: random_MPS, infinite_temp_MPO
using mVQE.Layers: Rylayer, CXlayer, Rxlayer
using mVQE.Circuits: runcircuit, VariationalCircuitRy, VariationalMeasurement, initialize_circuit, generate_circuit
using mVQE: loss, optimize_and_evolve
end

  0.030186 seconds (44.96 k allocations: 1.621 MiB, 15.85% compilation time)


In [3]:
N_state = 5
ancilla_frequency = 1
N = N_state * (ancilla_frequency + 1) - ancilla_frequency
ancillas_indices = [i for i in 1:N if mod1(i, ancilla_frequency+1)!=1]
state_indices = [i for i in 1:N if mod1(i, ancilla_frequency+1)==1];

In [4]:
function hamiltonian(state_indices, h)
    os = OpSum()
    for (i, s) in enumerate(state_indices)
        os += -1, "Z", s, "Z", state_indices[mod1(i+1, length(state_indices))]
        os += -h, "X", s
    end
    
    return os
end

hamiltonian (generic function with 1 method)

In [5]:
h = 1.  # transverse magnetic field

# Hilbert space
hilbert = qubits(N)

# build MPO "cost function"
H = MPO(hamiltonian(state_indices, h), hilbert);


hilbert_s = hilbert[state_indices]
H_s = MPO(hamiltonian(1:N_state, h), hilbert_s);

In [6]:
# Find ground state with DMRG

nsweeps = 20
maxdim = 2
cutoff_ = 1e-10

start_mps = randomMPS(hilbert, linkdims=2)
Edmrg, Φ = dmrg(H, start_mps; outputlevel=0, nsweeps, maxdim, cutoff=cutoff_);
println("Ground state energy from DMRG: $Edmrg")
maxlinkdim(Φ)

Ground state energy from DMRG: -6.414910943567371


2

In [7]:
start_mps = randomMPS(hilbert_s, linkdims=10)
Edmrg, Φ_state = dmrg(H_s, start_mps; outputlevel=0, nsweeps, maxdim, cutoff=cutoff_);
println("Ground state energy from DMRG: $Edmrg")

Ground state energy from DMRG: -6.413640314458521


In [8]:
# contract(Φ.data).tensor[:, 1, :]

## Random ansatz

In [8]:
depth = 6
model = VariationalCircuitRy(N, depth)
ψ = productstate(hilbert, fill(0, N))
θs = initialize_circuit(model);

In [9]:
ψout = runcircuit(ψ, model, θs, maxdim=2)
maxlinkdim(ψout)

2

## Variational ansatz

In [10]:
noise_ = (1 => ("depolarizing", (p = 1e-4,)), 
         2 => ("depolarizing", (p = 0.0,)))

Random.seed!(1234)

# run VQE using BFGS optimization
optimizer = LBFGS(; maxiter=50, verbosity=1)

LBFGS{Float64, HagerZhangLineSearch{Rational{Int64}}}(8, 50, 1.0e-8, true, HagerZhangLineSearch{Rational{Int64}}(1//10, 9//10, 1//1000000, 1//2, 2//3, 5//1, 9223372036854775807, -1), 1)

# With errors

In [9]:
optimizer = LBFGS(; maxiter=10, verbosity=1)

LBFGS{Float64, HagerZhangLineSearch{Rational{Int64}}}(8, 10, 1.0e-8, true, HagerZhangLineSearch{Rational{Int64}}(1//10, 9//10, 1//1000000, 1//2, 2//3, 5//1, 9223372036854775807, -1), 1)

In [15]:
#ρs = infinite_temp_MPO(hilbert)

ψ = productstate(hilbert, fill(0, N))
ρs = outer(ψ, ψ')

k = 2
depth = 5
model = VariationalCircuitRy(N, depth)
@time loss_value, θs_error, ρ, niter = optimize_and_evolve(k, ancillas_indices, ρs, H, model, depth; optimizer=optimizer, verbose=true, maxdims, noise=noise_);
ρ, = projective_measurement(ρ; indices=ancillas_indices, reset=1);

┌ Warning: LBFGS: not converged to requested tol: f = -2.824928724859, ‖∇f‖ = 2.3391e-02
└ @ OptimKit /Users/alcalde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:141


iter: 1
Loss: -2.824928724858896

iter: 2
Loss: -2.822275682606981



┌ Warning: LBFGS: not converged to requested tol: f = -2.822275682607, ‖∇f‖ = 3.8798e-02
└ @ OptimKit /Users/alcalde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:141


210.672968 seconds (445.25 M allocations: 23.668 GiB, 5.94% gc time, 98.89% compilation time)


# Optimize through the entire loop

In [ ]:
optimizer = LBFGS(; maxiter=100, verbosity=2)

ρ = infinite_temp_MPO(hilbert)
ρ, = projective_measurement(ρ; indices=ancillas_indices, reset=1)

ψ = productstate(hilbert, fill(0, N))
ρs = outer(ψ, prime(ψ))

depth = 5
model = VariationalCircuitRy(N, depth)
reset = 1
k = 4
model = VariationalMeasurement(model, k, ancillas_indices, reset)
@time loss_value, θs_error2, ρ2, niter = optimize_and_evolve(ρs, H, model; optimizer, noise=noise_, maxdim=50);
ρ2, = projective_measurement(ρ2; indices=ancillas_indices, reset=1);

In [38]:
θs = initialize_circuit(model);

In [39]:
loss(ρs, H, model, θs)

0.6758652705238788

In [202]:
inner(Φ, ρ2, Φ')

0.0002890192316163327 + 0.0im

# VQE

In [28]:
ψ = productstate(hilbert_s, fill(0, N_state))
ρs_s = outer(ψ, ψ')

model = VariationalCircuitRy(N_state, 8)


optimizer = LBFGS(; gradtol=1e-8, maxiter=150, verbosity=1)
@time loss_value, θs, ρ_s, niter = optimize_and_evolve(ρs_s, H_s, model, depth; optimizer=optimizer, verbose=true, maxdims, noise);
println(loss_value/Edmrg)

 50.640185 seconds (48.41 M allocations: 11.965 GiB, 5.95% gc time)
0.974617613731652


┌ Warning: LBFGS: not converged to requested tol: f = -6.307857700209, ‖∇f‖ = 1.1731e-02
└ @ OptimKit /home/leinad/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:141


In [204]:
inner(ρ_s, H_s)

-6.251523086913398 + 0.0im

In [205]:
inner(Φ_state, ρ_s, Φ_state')

0.5202533766943219 + 0.0im